In [11]:
import psycopg2
from config import load_config
import pandas as pd
import folium
import os

In [19]:
def fetch_data(conn, table_name, linha):
    cur = conn.cursor()
    all_data = [] 
    try:
        fetch_query = f'''
        SELECT latitude::double precision, longitude::double precision
        FROM {table_name}
        WHERE linha='{linha}'
        '''
        cur.execute(fetch_query)
        rows = cur.fetchall()
        all_data.extend(rows)
    except Exception as e:
            print(f"Erro ao executar a query na tabela {table_name}: {e}")
            conn.rollback()
    else:
         conn.commit()
    return all_data

In [13]:
def connect(config):
    """ Connect to the PostgreSQL database server """
    try:
        # connecting to the PostgreSQL server
        with psycopg2.connect(**config) as conn:
            print('Connected to the PostgreSQL server.')
            return conn
    except (psycopg2.DatabaseError, Exception) as error:
        print(error)


In [14]:
config = load_config()
conn = connect(config)

Connected to the PostgreSQL server.


In [15]:
linhas_de_interesse = ['483', '864', '639', '3', '309', '774', '629', '371', '397', '100', '838', '315', '624', '388', '918', '665', '328', '497', '878', '355', '138', '606', '457', '550', '803', '917', '638', '2336', '399', '298', '867', '553', '565', '422', '756', '186012003', '292', '554', '634', '232', '415', '2803', '324', '852', '557', '759', '343', '779', '905', '108']

In [16]:
df = pd.read_sql_query("SELECT * FROM dia_2704 WHERE linha='415'", conn)

/tmp/ipykernel_12976/231950575.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM dia_2704 WHERE linha='415'", conn)


In [17]:
df['ordem'].unique()

array(['A48047', 'A48090', 'A48140', 'A48032', 'A48099', 'A48040',
       'A48038', 'A48008', 'A48145', 'A48045', 'A48022', 'A48024',
       'A48125', 'A48075', 'A48119', 'A48036', 'A48066', 'A48010',
       'A48146', 'A48138', 'A48011', 'A48003', 'A48178', 'A48068',
       'A48071', 'A48149', 'A48122', 'A48007', 'A48087', 'A48113',
       'A48179', 'A48176', 'A48086', 'A48151', 'A48001'], dtype=object)

In [20]:
for linha in linhas_de_interesse:
    data = fetch_data(conn, 'dia_2704', linha)
    if data:
        avg_latitude = sum([d[0] for d in data]) / len(data)
        avg_longitude = sum([d[1] for d in data]) / len(data)
        m = folium.Map(location=[avg_latitude, avg_longitude], zoom_start=12)
        # Adicione os pontos ao mapa
        for lat, lon in data:
            folium.CircleMarker(location=[lat, lon], radius=1, color='blue').add_to(m)
        output_path = os.path.join(f'trajetos/output_map_{linha}.html')
        m.save(output_path)
        print(f"Mapa salvo em {output_path}")

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [8]:
conn.close()